# RAG CoT
(Chain of Thought)

RAG 파이프라인에서 LLM이 단순한 정보 조합을 넘어서 단계적 사고를 통해 논리적 답변을 할 수 있도록 한다.

In [4]:
%pip install langchain langchain-openai -Uq

Note: you may need to restart the kernel to use updated packages.


In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")

## 가상 Retriever

In [7]:
# 간단한 벡터 DB 입출력 예제
from langchain_core.documents import Document

def retrieve_vectordb(query=None):
    return [
        Document(page_content="대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다."),  # 서울 기본 정보
        Document(page_content="서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다."),  # 서울 관광 정보
        Document(page_content="서울의 인구는 약 천만 명이며, 교통·문화 인프라가 잘 갖춰져 있습니다.")  # 서울 인구/인프라 정보
    ]

retrieve_vectordb()

[Document(metadata={}, page_content='대한민국의 수도는 서울입니다. 서울은 한강을 끼고 발달한 도시입니다.'),
 Document(metadata={}, page_content='서울의 대표적 관광지는 경복궁, 남산타워, 명동 등이 있습니다.'),
 Document(metadata={}, page_content='서울의 인구는 약 천만 명이며, 교통·문화 인프라가 잘 갖춰져 있습니다.')]

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate.from_template('''  # 검색 문서 기반 답변 지시 프롬프트
당신은 데이터를 분석해서 논리적인 결론을 도출하는 전문가 챗봇입니다.
아래 [검색된 문서]를 바탕으로 사용자의 [질문]에 대해 답변하세요.

[검색된 문서]
{context}

[질문]
{question}

[지시사항]
다음의 단계에 따라 사고한 후, 답변을 작성하세요.
1. **핵심데이터 정리**: 문서에서 사용자 질문과 관련한 팩트를 추출해보세요.
2. **상호관계 분석**: 각 항목별로 어떤 상관관계/시너지를 도출하는지 고민하세요.
3. **논리적 서술**: 위의 사고한 내용을 토대로 사용자 질문에 대한 답변을 준비하세요.
4. **최종 답변**: 서론-본론-결론 구조에 맞춰 완성된 답변을 작성하세요.
''')

llm = init_chat_model('gpt-4.1-mini')  # 답변 생성용 LLM 설정
output_parser = StrOutputParser()      # LLM 출력 문자열로 파싱

chain = prompt | llm | output_parser

question = '서울의 인구, 관광지, 교육인프라를 종합해서 여행하기 좋은 이유를 논리적으로 설명해주세요.'

retrieved_docs = retrieve_vectordb(question)  # 더미 벡터 검색 수행
context = '\n\n'.join([doc.page_content for doc in retrieved_docs])  # retrieved_docs에서 page content만 빼서 하나의 텍스트로 병합

response = chain.invoke({'context': context, 'question': question})  # 컨텍스트 + 질문으로 답변 생성
print(response)

1. **핵심데이터 정리**  
- 서울의 인구: 약 천만 명  
- 서울의 관광지: 경복궁, 남산타워, 명동 등  
- 서울의 교육인프라: 문서에는 명시되어 있지 않으나, 서울은 대한민국의 중심지이자 문화·교통 인프라가 잘 갖춰진 도시임으로 교육 인프라도 발달했을 것으로 추정 가능  

2. **상호관계 분석**  
- 많은 인구는 다양한 서비스와 문화, 교육 인프라가 발달하는 기반이 됨  
- 풍부한 관광지는 방문객이 머무르며 여러 경험을 할 수 있게 하여 여행 만족도를 높임  
- 교육 인프라가 잘 갖춰져 있는 도시는 문화적 수준과 정보 접근성이 높아 여행객에게 다양한 체험과 편의를 제공함  
- 교통 인프라가 발달하여 관광지와 교육 시설 접근이 용이하다는 점 또한 긍정적 영향  

3. **논리적 서술**  
서울은 약 천만 명의 인구가 거주하는 대도시로서, 문화적 다양성과 서비스가 풍부합니다. 경복궁, 남산타워, 명동과 같은 유명 관광지는 서울만의 역사와 현대적 매력을 동시에 느낄 수 있게 해줍니다. 또한, 서울은 교통과 문화 인프라뿐만 아니라 교육 인프라도 잘 갖춰져 있어서, 다양한 문화 행사와 체험 기회를 제공합니다. 이러한 요소들은 서로 시너지를 이루어 방문객들에게 편리하고 풍부한 여행 경험을 제공하며, 서울이 여행하기 좋은 도시임을 뒷받침합니다.  

4. **최종 답변**  
서울은 약 천만 명의 인구가 모여 살아가는 대도시로, 이로 인해 다양한 문화와 서비스가 활발하게 이루어지고 있습니다. 대표 관광지인 경복궁, 남산타워, 명동 등은 서울의 역사적 가치와 현대적 아름다움을 동시에 체험할 수 있는 장소로, 방문객들에게 다채로운 볼거리와 즐길 거리를 선사합니다. 더불어, 서울은 훌륭한 교육 인프라와 함께 교통·문화 인프라도 잘 발달해 있어 여행자들이 편리하게 이동하며 풍부한 문화적 경험을 할 수 있습니다. 이처럼 서울의 인구, 관광지, 교육 인프라가 상호 보완적으로 작용하여 쾌적하고 의미 있는 여행을 가능하게 만들기 때문에 서울은 여행하기에 매